# Symbolic calculations in Julia

`Symbolics.jl` is a computer Algebra System (CAS) for Julia. The symbols are number-like and follow Julia semantics so we can put them into a regular function to get a symbolic counterpart.

Source:
- [Simulating Big Models in Julia with ModelingToolkit @ JuliaCon 2021 Workshop](https://youtu.be/HEVOgSLBzWA).
- [Symbolics.jl](https://github.com/JuliaSymbolics/Symbolics.jl) Github repo and its [docs](https://symbolics.juliasymbolics.org/dev/).

## Caveats about Symbolics.jl

`Symbolics.jl` can only handle *traceble*, *quasi-static* expressions.

Some code is not quasi-static e.g. factorial. The number of operations depends on the input value.

> Use `@register` to make it a primitive function.

Some code is *untraceable* like conditional `if`...`else` statements.

> You can use `ifelse(cond, ex1, ex2)` to make it traceable.

## Basic operations

In [ ]:
using Symbolics

In [ ]:
# Symbolics
@variables x y

In [ ]:
x^2 + y^2

In [ ]:
# You can use Latexify.jl to generate latex code
A = [x^2 + y 0 2x
     0       0 2y
     y^2 + x 0 0]

In [ ]:
Symbolics.derivative(x^2 + y^2, x)

In [ ]:
Symbolics.gradient(x^2 + y^2, [x, y])

In [ ]:
Symbolics.jacobian([x^2 + y^2; y^2], [x, y])

In [ ]:
Symbolics.substitute(sin(x)^2 + 2 + cos(x)^2, Dict(x=>y^2))

In [ ]:
Symbolics.substitute(sin(x)^2 + 2 + cos(x)^2, Dict(x=>1.0))

In [ ]:
Symbolics.simplify(sin(x)^2 + 2 + cos(x)^2)

In [ ]:
# Automatically simplify because it's always true
2x - x

In [ ]:
ex = x^2 + y^2 + sin(x)

In [ ]:
isequal(2ex, ex + ex)

In [ ]:
ex / ex

In [ ]:
x / expm1(x)

## Custom functions

In [ ]:
foo(x, y) = x * rand() + y

Without @register, the `rand()` in `foo()` will be evaulated numerically instead of symbolically.

> By default, new functions are traced to the primitives and the symbolic expressions are written on the primitives. However, we can expand the allowed primitives by registering new functions.

In [ ]:
# Bring foo(() into symbolic land
@register foo(x, y)

In [ ]:
Symbolics.derivative(foo(hypot(x, y), y), x)

## More number types

In [ ]:
@variables z::Complex

In [ ]:
@variables xs[1:18]

In [ ]:
xs[1]

In [ ]:
# Lazy representation of summation
sum(xs)

In [ ]:
# Explicit vector form
collect(xs)

In [ ]:
# Show explicit formulae
sum(collect(xs))

## Example: Rosenbrock function

Wikipedia: <https://en.wikipedia.org/wiki/Rosenbrock_function>

In [ ]:
# The vector form of Rosenbrock function
rosenbrock(xs) = sum( 1:length(xs)-1) do i
    100*(xs[i+1] - xs[i]^2)^2 + (1 - xs[i])^2
end

In [ ]:
# Minimum when xs are all one's
rosenbrock(ones(100))

In [ ]:
N = 100
@variables xs[1:N]

In [ ]:
# A long list of vector components
xs = collect(xs)

In [ ]:
rxs = rosenbrock(xs)

In [ ]:
grad = Symbolics.gradient(rxs, xs)

In [ ]:
# hes = Symbolics.jacobian(grad, xs)
hes = Symbolics.hessian(rxs, xs)

In [ ]:
isequal(Symbolics.hessian(rxs, xs), Symbolics.jacobian(grad, xs))

## Sparse matrix

In [ ]:
# Sparse Hessian matrix of the Hessian matrix of the Rosenbrock function w.r.t. to vector components.
# (298 non-zero values out of 10000 elements)
hes_sp = Symbolics.hessian_sparsity(rxs, xs)

Visualize sparse matrix using `Plots.spy()`

In [ ]:
# Visualizing sparse matrix using Plots.spy()
using Plots
spy(hes_sp)

In [ ]:
using BenchmarkTools
@benchmark Symbolics.hessian($rxs, $xs)

In [ ]:
# Sparse is faster
@benchmark Symbolics.hessian_sparsity($rxs, $xs)

## Generating functions

- `build_function(ex, args...)` generates out-of-place (oop) and in-place (ip) function expressions in a pair.
- `build_function(ex, args..., parallel=Symbolics.MultithreadedForm())` generates a parallel algorithm to evaluate the output. See the [example](https://symbolics.juliasymbolics.org/dev/tutorials/symbolic_functions/#Building-Non-Allocating-Parallel-Functions-for-Sparse-Matrices-1) in the official docs. **However**, there is [an issue](https://github.com/JuliaSymbolics/Symbolics.jl/issues/136) blocking it from happending.
- `build_function(ex, args..., target=Symbolics.CTarget())` generates a C function from Julia.

In [ ]:
fexprs = build_function(grad, xs);

In [ ]:
foop = eval(fexprs[1])

In [ ]:
fip = eval(fexprs[2])

In [ ]:
inxs = rand(N)
out = similar(inxs);

In [ ]:
fip(out, inxs)  # The inplace version returns nothing. The results are stored in out.

In [ ]:
foop(inxs)

In [ ]:
# Check answers
foop(inxs) ≈ out

In [ ]:
# Save the generated function for later use
# write("function.jl", string(fexprs[2]))

We can use another package `ForwardDiff.jl` to make sure our gradient from `Symbolics.jl` is correct.

In [ ]:
using ForwardDiff
ForwardDiff.gradient(rosenbrock, inxs) ≈ out

In [ ]:
# Sparce Hessian matrix
# Only non-zero expressions are calculated
hexprs = build_function(hes_sp, xs);

In [ ]:
hoop = eval(hexprs[1])
hip = eval(hexprs[2])

In [ ]:
hoop(rand(N))